In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
device = "cuda"
model_name_or_path = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
tokenizer_name_or_path = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"

In [3]:
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

In [4]:
dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)

In [5]:
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [26]:
dataset = load_dataset("ought/raft", dataset_name)

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})

In [28]:
dataset['train'][0]

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [29]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)

In [30]:
dataset['train'][0]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[ # type: ignore
            "attention_mask"
        ][i] # type: ignore
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [11]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())
#"trainable params: 8192 || all params: 559222784 || trainable%: 0.0014648902430985358"

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

trainable params: 16,384 || all params: 1,235,830,784 || trainable%: 0.0013
None


In [14]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

In [15]:
training_args = TrainingArguments(
    output_dir='/Utilisateurs/umushtaq/emotion_analysis_comics/softprompt_models',
    use_cpu=False,
    auto_find_batch_size=False,
    learning_rate=1e-4,
    logging_steps=100,
    per_device_train_batch_size=1,
    save_total_limit=1,
    remove_unused_columns=False,
    num_train_epochs=5,
    fp16=True,
    save_strategy='no',
    logging_dir="logs",
    report_to="none"
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Custom CPT training dataset.
    eval_dataset=eval_dataset
    #data_collator=CPTDataCollatorForLanguageModeling(tokenizer, training=True, mlm=False)
)

In [17]:
trainer.train()

Step,Training Loss
100,12.364200
200,6.929900


TrainOutput(global_step=250, training_loss=8.851354736328124, metrics={'train_runtime': 25.1535, 'train_samples_per_second': 9.939, 'train_steps_per_second': 9.939, 'total_flos': 93422026752000.0, 'train_loss': 8.851354736328124, 'epoch': 5.0})

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
peft_model_id = "mohammadomar/llama3.1_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("mohammadomar/llama3.1_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/65.7k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mohammadomar/llama3.1_PROMPT_TUNING_CAUSAL_LM/commit/95f69b2a002e23d94dbce0632d5d7a030d74e080', commit_message='Upload model', commit_description='', oid='95f69b2a002e23d94dbce0632d5d7a030d74e080', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mohammadomar/llama3.1_PROMPT_TUNING_CAUSAL_LM', endpoint='https://huggingface.co', repo_type='model', repo_id='mohammadomar/llama3.1_PROMPT_TUNING_CAUSAL_LM'), pr_revision=None, pr_num=None)

In [20]:
from peft import PeftModel, PeftConfig

In [21]:
peft_model_id = "mohammadomar/llama3.1_PROMPT_TUNING_CAUSAL_LM"

In [22]:
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/542 [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


adapter_model.safetensors:   0%|          | 0.00/65.7k [00:00<?, ?B/s]

In [23]:
inputs = tokenizer(
    f'{text_column} : {"@nationalgridus I have no water and the bill is current and paid. Can you do something about this?"} Label : ',
    return_tensors="pt",
)

In [24]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/peft/peft_model.py:1889: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


['Tweet text : @nationalgridus I have no water and the bill is current and paid. Can you do something about this? Label : Question']
